## Прогнозирование откликов на скидки

In [23]:
import pandas as pd

from typing import NamedTuple

from pandas import DataFrame
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

Данные

In [2]:
df = pd.read_csv('../../data/offer_responses.csv')
df.head()

,responses,offer_discount,offer_quality,offer_reach
0,4151.0,26.0,10.257680,31344.0
1,3397.0,35.0,15.194380,24016.0
2,3274.0,21.0,13.971468,28832.0
3,3426.0,27.0,6.054338,26747.0
4,5745.0,42.0,16.801365,46968.0


In [3]:
X = df[['offer_quality',
        'offer_discount',
        'offer_reach']]
 
y = df['responses']

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 10)

In [22]:
type(X_train)

pandas.core.frame.DataFrame

Обучение модели для всех предикторов

In [7]:
model = LinearRegression()
model.fit(X_train,y_train)

predictions = model.predict(X_test)

print(f'RMSE with all variables: {str(mean_squared_error(predictions, y_test)**0.5)}')

RMSE with all variables: 966.2461828577945


In [29]:
class RSMEWithoutOffer(NamedTuple):
    without_column: str
    rsme: float

RSME минимальна при отсутствии offer_quality, поэтому этот параметр можно отбросить

In [31]:
def get_RMSE(X_train: DataFrame,
             X_test: DataFrame,
             y_train: DataFrame,
             y_test: DataFrame) -> RSMEWithoutOffer:
    for column in X.columns:
        X_train2 = X_train.drop(column, axis=1)
        X_test2 = X_test.drop(column, axis=1)

        model = LinearRegression()
        model.fit(X_train2, y_train)

        predictions = model.predict(X_test2)

        yield RSMEWithoutOffer(column, mean_squared_error(predictions, y_test)**0.5)


for column, rsme in get_RMSE(X_train, X_test, y_train, y_test):
    print(f'RMSE without offer {column}: {rsme}')


RMSE without offer offer_quality: 965.5346123758474
RMSE without offer offer_discount: 1231.6766556327284
RMSE without offer offer_reach: 1185.8456831644116
